In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M").to(device)

/home/leloc/anaconda3/envs/translate/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


KeyboardInterrupt: 

In [7]:
import duckdb

con = duckdb.connect("/home/leloc/Document/USTH/Thesis/Data/translation.db")
df = con.execute("SELECT * FROM translations LIMIT 100").fetchdf()


In [8]:
df

,thai,viet,thai_input_ids,thai_attention_mask,vi_input_ids,vi_attention_mask
0,- รร.แกรนด์โฮเต็ล,- Khách sạn Grand.,"[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2..."
1,- สายไม่ว่าง,- Đường dây bận.,"[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2..."
2,นั่นใครพูดคะ,Ai đấy?,"[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2..."
3,ฉันจะโอนสายคุณ ไปที่แผนกรูมเซอร์วิส,Tôi có thể kết nối quý khách với dịch vụ phòng.,"[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2..."
4,ฮัลโหลๆ,A lô.,"[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2..."
...,...,...,...,...,...,...
95,รอเหรอ ผมรอข่าว จากทางแมนเชสเตอร์นานแล้ว,Chờ? Tôi đã chờ đợi tin tức từ Manchester.,"[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2..."
96,- ไม่มีข่าวเลยเหรอ,- Không có tin gì?,"[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2..."
97,- ไม่มี,- Không.,"[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2..."
98,แย่แล้ว ถ้าคุณรวมกิจการ กับบ.แซกโซเนียไม่ได้ล่...,Tệ thật. Anh không thể cầm cự trừ khi anh hợp ...,"[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2...","[128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2..."


In [9]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm import tqdm
from pythainlp.util import count_thai_chars
import sacrebleu

# -----------------------
# 1. Load mô hình & tokenizer
# -----------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    src_lang="tha_Thai",
    tgt_lang="vie_Latn"
)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# -----------------------
# 2. Load dữ liệu và lọc chỉ giữ tiếng Thái
# -----------------------
# Giả sử bạn đã có df chứa các cột 'thai' và 'viet'
# df = pd.read_csv("duong_dan_toi_file.csv")

# Lọc dữ liệu có ít nhất 50% ký tự là tiếng Thái
df = df[df["thai"].apply(lambda x: sum(count_thai_chars(x).values()) / len(x) > 0.5)]

# -----------------------
# 3. Dịch từ Thái sang Việt
# -----------------------
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    output = model.generate(**inputs)
    return tokenizer.decode(output[0], skip_special_tokens=True)

tqdm.pandas()
df["viet_predicted"] = df["thai"].progress_apply(translate)

# -----------------------
# 4. Tính điểm BLEU với sacrebleu
# -----------------------
references = df["viet"].tolist()
hypotheses = df["viet_predicted"].tolist()

bleu_score = sacrebleu.corpus_bleu(hypotheses, [references])
print("🎯 BLEU (sacrebleu):", bleu_score.score)

# -----------------------
# 5. Hiển thị ví dụ ngẫu nhiên
# -----------------------
print(df[["thai", "viet", "viet_predicted"]].sample(5))


cuda


100%|██████████| 100/100 [12:02<00:00,  7.22s/it]

🎯 BLEU (sacrebleu): 0.5290856235116009
                                          thai  \
87            เมื่อคืนคุณอยู่ที่โรงละครรึเปล่า   
16            ไม่ มีคนมาบอกอย่างนี้มันดีตรงไหน   
18               ฉันอยากทำอะไรสักอย่างกับชีวิต   
8   นี่อ็อตโต้ คริงเกลไลน์ นั่นนายเหรอ ไฮน์ริช   
7                   อดทนเหรอ เป็นคุณจะทนไหมล่ะ   

                                                 viet  \
87                        Anh đã đến nhà hát tối qua?   
16  Không, thật tuyệt khi được nói những điều như ...   
18               Tôi muốn có được gì đó từ cuộc sống.   
8           Otto Kringelein đây. Anh đấy à, Heinrich?   
7                  Kiên nhẫn? Anh có kiên nhẫn không?   

                                       viet_predicted  
87                Were you at the theater last night?  
16  No, no, no, no. No, no, no. No, no, no. No, no...  
18             I wanted to do something with my life.  
8              This is Otto Kringlelain. Is that you?  
7                                    